In [7]:
#hide
from icecream import ic
import sys, re

def jupyter(*args): 
    print(*[re.sub(r",\s{1,}", ", ", i.replace(",\n", ", ")) for i in args], file=sys.stdout)
    
ic.configureOutput(prefix='ic> ', outputFunction=jupyter)


import functools

## todo

In [ ]:
# reprlib.recursive_repr

# Decorators

[PEP 3129](https://www.python.org/dev/peps/pep-3129/), [PEP 318](https://www.python.org/dev/peps/pep-0318/)

### Resources
* [PythonWiki: PythonDecorators](https://wiki.python.org/moin/PythonDecorators)
* [PythonWiki: PythonDecoratorLibrary](https://wiki.python.org/moin/PythonDecoratorLibrary)
* [Stackoverflow: How to make a chain of function decorators?
](https://stackoverflow.com/questions/739654/)
* https://github.com/lord63/awesome-python-decorator

### Talks
* [PyCon2019: Practical decorators](https://youtu.be/MjHpMCIvwsY) by Reuven M. Lerner - [slides](https://speakerdeck.com/pycon2019/reuven-m-lerner-practical-decorators) 
* [EuroPytop 2018: A Taxonomy of Decorators: A-E](https://youtu.be/pEL1THG6ysY) by Andy Fundinger - [slides](https://github.com/Ciemaar/decorator-taxonomy)

### Example

In [2]:
def decorator(func):
    def wrapper(*args, **kwargs):
        return f"{func(*args, **kwargs)}"
    
    return wrapper

@decorator
def add(a, b):
    return a+b

ic(add(1,2))

ic> add(1,2): '3'


'3'

## Decorators with params

In [3]:
def decorator(func, argument):
    def wrapper(number):
        return func(number+argument)
    return wrapper

more_at_5 = functools.partial(decorator, argument=5)

@more_at_5
def pow(a):
    return a**2

ic(pow(5))

ic> pow(5): 100


100

### Using `Class` as a decorator

In [4]:
class slim_shady: 
    
    def __init__(self, func):
        self.func = func
        
    def __call__(self, name): 
        return self.func(self.__class__.__name__.replace("_", " "))
    
@slim_shady
def name(name):
    return "My name is {}".format(name)

ic(name('oleg'))

ic> name('oleg'): 'My name is slim shady'


'My name is slim shady'

or using a wrapper inside wrapper inside decorator

## `functools`'s decorators

`functools.lru_cache` - caching for LRU objects/calls

In [5]:
def fib1(n): 
    return n if n in (0, 1) else (fib1(n-1)+fib1(n-2))

@functools.lru_cache(maxsize=35)
def fib2(n): 
    return n if n < 2 else (fib2(n-1)+fib2(n-2))

%timeit fib1(35)
%timeit fib2(35)

# cache info
ic(fib2.cache_info())

6.23 s ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
117 ns ± 0.485 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
ic> fib2.cache_info(): CacheInfo(hits=81111143, misses=36, maxsize=35, currsize=35)


CacheInfo(hits=81111143, misses=36, maxsize=35, currsize=35)

####  `functools.wraps` - allows to mask wrapper function name

* [What does `functools.wraps` do?](https://stackoverflow.com/questions/308999)

In [6]:
def loud(func): 
    def wrapper(*args, **kwargs):
        args = map(lambda x: x.upper(), (args))
        return func(*args, **kwargs)
    return wrapper

@loud
def mprint(string):
    return string


ic(mprint.__name__)
ic(mprint("yo"))

# ------------------------------------------------- 
del mprint

# -------------------------------------------------
def wisper(func): 
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args = map(lambda x: x.lower(), (args))
        return func(*args, **kwargs)
    return wrapper

@wisper
def mprint(string):
    return string

ic(mprint.__name__)
ic(mprint("YO"))

ic> mprint.__name__: 'wrapper'
ic> mprint("yo"): 'YO'
ic> mprint.__name__: 'mprint'
ic> mprint("YO"): 'yo'


'yo'

#### `functools.total_ordering` 

Given a class defining one or more rich comparison ordering methods, this class decorator supplies the rest. This simplifies the effort involved in specifying all of the possible rich comparison operations:

The class must define one of `__lt__()`, `__le__()`, `__gt__()`, or `__ge__()`. In addition, the class should supply an `__eq__()` method.

In [7]:
@functools.total_ordering
class Weight:
    def __init__(self, weight:str):
        self.number, self.units = weight.split(" ") 
    
    @property
    def value(self):
        return self.converts_to_kilograms()
    
    def converts_to_kilograms(self):
        mesurments = {}
        mesurments['gr'] = 0.001
        mesurments['kg'] = 1 
        mesurments['quintal'] = 100
        mesurments['pound'] = 0.5
        mesurments['ton'] = 1000
        
        return float(self.number)*mesurments.get(self.units)
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __eq__(self, other):
        return self.value == other.value
    
ic(Weight('5 kg') == Weight('5000 gr'))
ic(Weight('5 kg') <  Weight('4999 gr'))
ic(Weight('5 kg') >  Weight('4999 gr'))

ic> Weight('5 kg') == Weight('5000 gr'): True
ic> Weight('5 kg') < Weight ('4999 gr'): False
ic> Weight('5 kg') > Weight ('4999 gr'): True


True

### `functools.singledispatch`

In [12]:
from typing import List

list_of_ints = List[int]

@functools.singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

# and overloading fucntions
@fun.register
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)
    
@fun.register(list)
def _(arg, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

import collections
@fun.register(collections.abc.Sequence)
def _(arg, verbose=False):
    if verbose:
        print("Sequence:")
    for i, elem in enumerate(arg):
        print(i, elem)

In [9]:
fun("its not so funny", True)

Let me just say, its not so funny


In [10]:
fun(12312, True)

Strength in numbers, eh? 12312


In [13]:
fun([123,12,3,123,12,3,12,3], True)

Enumerate this:
0 123
1 12
2 3
3 123
4 12
5 3
6 12
7 3


In [14]:
# list available implementations
ic(fun.registry.keys())

# --- checking implementations....

# default implementation
ic(fun.dispatch(str))
ic(fun.dispatch(float))
# existing implementation
ic(fun.dispatch(int))

ic> fun.registry.keys(): dict_keys([<class 'object'>, <class 'int'>, <class 'list'>, <class 'collections.abc.Sequence'>])
ic> fun.dispatch(str): <function _ at 0x10d648ae8>
ic> fun.dispatch(float): <function fun at 0x10d648ea0>
ic> fun.dispatch(int): <function _ at 0x10d6481e0>


<function __main__._(arg: int, verbose=False)>

### `@decorators` used in Object-Oriented Programming

* `@statickmethod` - static methods
* `@classmethod` - class creation
* `@abstractmethod` - static methods 


Note about `@abstractmethod`, in most cases you will use method that raises `NotImplemented` exception **only**^ which is not `abstract method`!

In [13]:
from abc import abstractmethod

class AbstractPower:
    @abstractmethod
    def power(number, power): 
        raise NotImplementedError("ddd")

class Power(AbstractPower):
    
    def __init__(self, number, power):
        self.number = number
        self.power = power
        
    @classmethod
    def cube(cls, number):
        return cls(number, 4)
    
    @classmethod
    def root(cls, number):
        return cls(number, 0.5)

    @staticmethod
    def power(x, power):
        return x**power
    
    def __repr__(self):
        return "{}({}^{}) is {}".format(
            self.__class__.__name__, 
            self.number,
            self.power,
            self.__class__.power(self.number, self.power)
        )
    
ic(Power(2, 2))
ic(Power.cube(2))
ic(Power.root(4)) 
ic(Power.cube(2))

ic> Power(2, 2): Power(2^2) is 4
ic> Power.cube(2): Power(2^4) is 16
ic> Power.root(4): Power(4^0.5) is 2.0
ic> Power.cube(2): Power(2^4) is 16


Power(2^4) is 16

### Getters and Setters

In [14]:
class Root:
    def __init__(self, number):
        self._n = number
    
    @property
    def n(self):
        return self._n
    
    @n.setter
    def n(self, n):
        self._n = n
        
    @property
    def root(self):
        return self._n * .5

root = Root(4)
ic(root)
ic(root.n)
ic(root.root)
root.n=16
ic(root.n)
ic(root.root)

ic> root: <__main__.Root object at 0x10f1ef630>
ic> root.n: 4
ic> root.root: 2.0
ic> root.n: 16
ic> root.root: 8.0


8.0

### `@dataclass`

In [8]:
import dataclasses

@dataclasses.dataclass
class Point:
    x: float
    y: float
    z: float = 0.0
 
ic(Point(1.5, 2.5))

ic> Point(1.5, 2.5): Point(x=1.5, y=2.5, z=0.0)


Point(x=1.5, y=2.5, z=0.0)

### Context Managers and `@contextmanager`

You can make your own context manager without `__enter__` and `__exit__`

In [15]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

with managed_file('hello.txt') as f:
    f.write('¡hola!')

### Exit function

In [16]:
import atexit

@atexit.register
def goodbye():
    print("Ciao...")
    
exit(0)

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


Ciao...


In [3]:
import sys

print(sys.version)

3.7.3 (default, Mar 27 2019, 09:23:39) 
[Clang 10.0.0 (clang-1000.11.45.5)]
